# **Week 2 -  Experiment Tracking**
**- ML Flow setup and testing...** \
\
This notebook is following the coursework for week 02....


In [5]:
import pickle

# Impoirting ML flow and then setting up the pipeline...
import mlflow
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

print('Done')

Done


In [15]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/15 19:13:00 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/martvas/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747354380186, experiment_id='1', last_update_time=1747354380186, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
# Defining a quick function for ingesting data...
def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = round((df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60, 3)
    df = df[(df['duration'] > 1) & (df['duration'] <= 60)]
    categorical  = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [9]:
df_train = read_data('./data/green_tripdata_2021-01.parquet')
df_val = read_data('data/green_tripdata_2021-02.parquet')
print(f'df_train data shape: {df_train.shape}')
print(f'df_val data shape: {df_val.shape}')

df_train data shape: (73833, 21)
df_val data shape: (61889, 21)


In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PuLocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred, squared=False)
print('Mean Squared Error: ', mse_val)

Mean Squared Error:  7.756930872164769


In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
# Testing a lasso model to see if we have improvement
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mse_lasso = mean_squared_error(y_val, y_pred, squared=False)
print('$$ Lasso Root Mean Squared Error: ', mse_lasso)

$$ Lasso Root Mean Squared Error:  11.163246810144308


### **Section 2.2. - Implementing MLFlow for tracking experiments:**

For this we can use the Lasso model for testing and showcasing... timestamp: **6:58** in this section the course went through:
- Basic logging for identyifigyin key parameters; i.e, parameters, data paths, metrics.
- Parametric logging for timing, versions and tagging.

In [19]:
# Testing a lasso model to see if we have improvement
with mlflow.start_run():

    #Logging some key info into mlflow.....
    mlflow.set_tag('developer', 'bald_fish')
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    

### **Section 2.3 - Experiment Tracking with ML flow.....**

In [ ]:
# Section 2.3. Adding hyperoptimization.
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope